## 1. 컬럼 이름 변환 및 정렬
- `UserVitalSign`과 `UserLocationLog` 데이터셋의 컬럼명을 **대문자로 변환**하여 일관성을 맞춤
- `UserGyro`의 컬럼 명은 샘플 데이터를 기반으로 `No`, `WorkDate`, `UserCode`, `X`, `Y`, `Z`, `VitalDate`, `RegisterDate`로 설정 가능
- `WorkDate`와 `VitalDate`의 날짜가 다를 수 있기 때문에 데이터 통합 과정에서 이를 보정하도록 함

## 2. 불필요한 열 제거
- 각 데이터셋에 프로젝트 구현에 필요하지 않은 열이 있는지 검토하고 제거함. 예를 들어, `UserLocationLog`에서 `coordinate`, `registerempcode`, `modifyempcode` 등 프로젝트의 목적에 맛지 않는 열을 삭제 가능
- 사용하지 않는 열을 미리 제거하여 데이터의 크기를 줄이고 분석 속도를 높임

## 3. 결측치 및 이상치 처리
- **결측치 확인**: 각 데이터셋에서 결측치가 있는 열을 식별함. 예를 들어, `heartbeat`, `temperature` 등의 생체 신호 컬럼에서 결측치가 발견될 경우 이를 보간하거나 제거 가능
- **이상치 처리**: 특정 값이 비정상적으로 높거나 낮은 경우 이를 이상치로 간주 가능. 예를 들어, `speed`나 `heartbeat`가 극단적인 값을 가지는 경우, 해당 값이 측정 오류인지 검토하여 대체 또는 제거함

## 4. 날짜 및 시간 통합
- `WorkDate`와 `VitalDate`의 날짜가 다른 문제를 해결하기 위해, `VitalDate`의 날짜 정보를 기반으로 데이터를 그룹화하여 분석을 진행함. 날짜 형식이 맞지 않거나 시계열 정렬이 필요할 경우, `VitalDate`를 기준으로 정렬해 데이터를 통일함
- 날짜 및 시간 컬럼을 통일하여 각 데이터셋에서 동일한 시간 범위에 맞는 데이터를 추출할 수 있도록 함

## 5. 데이터 형식 및 범위 검토
- **데이터 형식 통일**: `latitude`, `longitude`와 같은 열은 `float` 타입으로 통일하고, `UserCode`, `WorkDate`는 `string` 타입으로 처리하여 데이터 일관성을 유지함
- **범위 확인**: 각 변수의 값이 예상된 범위를 벗어나지 않는지 확인하여, 비정상적 값이 포함된 경우 수정함. 예를 들어, `heartbeat`의 경우 일반적인 심박수 범위를 설정하여 그 밖의 값은 대체하거나 제거함

## 6. 정제 후 데이터 저장
- 정제된 데이터를 새로운 CSV 파일로 저장하여 향후 분석과 예측 모델 학습에 사용할 수 있도록 준비함
- 정제 후 각 데이터셋의 통계 요약을 확인하고, 주요 변수의 분포를 분석하여 데이터 품질을 점검함

### 파일 불러오기, 컬럼 명 변환, 불필요한 열 제거, 결측치 및 이상치 처리 등의 작업을 단계별로 진행

In [5]:
import pandas as pd
from tqdm import tqdm

# tqdm 적용을 위한 설정 (pandas에서 자동으로 tqdm 적용)
tqdm.pandas()

# 1. 데이터 불러오기
print("데이터 로딩 중...")
gyro = pd.read_csv(r"C:\AI\원본 데이터\원본\UserGyro\BiscoDB_UserGyro-20240801.csv")
vital_sign = pd.read_csv(r"C:\AI\원본 데이터\원본\UserVitalSign.csv")
location_log = pd.read_csv(r"C:\AI\원본 데이터\원본\UserLocationLog.csv")
print("데이터 로딩 완료!")

# 2. 컬럼명 대문자로 변환
print("컬럼명을 대문자로 변환 중...")
vital_sign.columns = [col.upper() for col in vital_sign.columns]
location_log.columns = [col.upper() for col in location_log.columns]

# UserGyro 컬럼 이름 정의
gyro.columns = ["NO", "WORKDATE", "USERCODE", "X", "Y", "Z", "VITALDATE", "REGISTERDATE"]
print("컬럼명 변환 완료!")

# 3. 불필요한 열 제거
print("불필요한 열 제거 중...")
# NETWORKOPERATOR와 NETWORKPROVIDER 열은 결측치가 많으므로 삭제
location_log.drop(columns=["NETWORKOPERATOR", "NETWORKPROVIDER"], inplace=True)

vital_sign.drop(columns = ["AGENCYCODE", "APMAC", "DEVICEMAC", "REGISTEREMPID", "MODIFYEMPID"], inplace=True)
location_log.drop(columns = ["COORDINATE", "REGISTEREMPCODE", "MODIFYEMPCODE"], inplace=True)
print("불필요한 열 제거 완료!")

# 4. 결측치 처리
# 결측치 개수 확인
print("VitalSign 결측치 개수:\n", vital_sign.isnull().sum())
print("LocationLog 결측치 개수:\n", location_log.isnull().sum())
print("Gyro 결측치 개수:\n", gyro.isnull().sum())

# 결측치처리: 앞의 값으로 대체 (Forward Fill)
print("결측치 처리 중...")
vital_sign.ffill(inplace=True)
location_log.ffill(inplace=True)
gyro.ffill(inplace=True)
print("결측치 처리 완료!")

# 5. 이상치 처리
# 각 열의 이상치(예: 3 표준편차 이상) 탐지, 제거하지 않고 분석에 포함
# 이상치를 식별하고 마킹 (제거하지 않음)
print("이상치 탐지 및 마킹 중...")
def mark_outliers(df, column):
    mean = df[column].mean()
    std = df[column].std()
    df['{}_OUTLIER'.format(column)] = (df[column] > mean + 3 * std) | (df[column] < mean - 3 * std)

# tqdm을 사용하여 각 컬럼별로 이상치 마킹
for column in tqdm(['HEARTBEAT', 'TEMPERATURE', 'SPEED']):
    if column in vital_sign.columns:
        mark_outliers(vital_sign, column)
    if column in location_log.columns:
        mark_outliers(location_log, column)

print("이상치 마킹 완료!")

# 6. 날짜 및 시간 형식 변환 및 정렬
# datetime 형식 변환
print("날짜 및 시간 형식 변환 중...")
vital_sign['VITALDATE'] = pd.to_datetime(vital_sign['VITALDATE'], errors='coerce')
location_log['CHECKTIME'] = pd.to_datetime(location_log['CHECKTIME'], errors='coerce')
gyro['VITALDATE'] = pd.to_datetime(gyro['VITALDATE'], errors='coerce')

# 변환 실패한 날짜는 결측치로 처리 후 제거
vital_sign.dropna(subset=['VITALDATE'], inplace=True)
location_log.dropna(subset=['CHECKTIME'], inplace=True)
gyro.dropna(subset=['VITALDATE'], inplace=True)

# UserGyro에서 VITALDATE의 날짜 정보로 WORKDATE 갱신
gyro['WORKDATE'] = gyro['VITALDATE'].dt.strftime('%Y%m%d')

print("날짜 및 시간 변환 완료!")

# 7. 각 데이터셋 오름차순 정렬
# VITALDATE, CHECKTIME, VITALDATE 기준으로 과거 순서대로 정렬
vital_sign.sort_values(by='VITALDATE', inplace=True)
location_log.sort_values(by='CHECKTIME', inplace=True)
gyro.sort_values(by='VITALDATE', inplace=True)

# 각 데이터셋의 첫 번째 5개 행 확인
print("UserVitalSign 데이터:\n", vital_sign.head())
print("UserLocationLog 데이터:\n", location_log.head())
print("UserGyro 데이터:\n", gyro.head())

# 정제된 데이터 저장
print("정제된 데이터 저장 중...")
vital_sign.to_csv("Cleaned_UserVitalSign.csv", index=False)
location_log.to_csv("Cleaned_UserLocationLog.csv", index=False)
gyro.to_csv("Cleaned_UserGyro.csv", index=False)
print("정제된 데이터 저장 완료!")

print("모든 전처리 작업이 완료되었습니다!")

데이터 로딩 중...
데이터 로딩 완료!
컬럼명을 대문자로 변환 중...
컬럼명 변환 완료!
불필요한 열 제거 중...
불필요한 열 제거 완료!
VitalSign 결측치 개수:
 NO                    0
WORKDATE              0
USERCODE              0
ISWEAR                0
HEARTBEAT             0
TEMPERATURE           0
OUTSIDETEMPERATURE    0
LATITUDE              0
LONGITUDE             0
DEVICEBATTERY         0
VITALDATE             0
REGISTERDATE          0
MODIFYDATE            0
dtype: int64
LocationLog 결측치 개수:
 NO                          0
WORKDATE                    0
USERCODE                    0
LATITUDE                    0
LONGITUDE                   0
SATELLITE                   0
ACCURACY                    0
ALTITUDE                    0
VERTICALACCURACY            0
BEARING                     0
BEARINGACCURACY             0
DIRECTION                   0
SPEED                       0
SPEEDACCURACY               0
UNIXTIME                    0
CHECKTIME                   0
LOCATIONPROVIDER            0
ELAPSEDREALTIMEAGEMILLIS    0
REGISTERDATE  

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 39.99it/s]

이상치 마킹 완료!
날짜 및 시간 형식 변환 중...


날짜 및 시간 변환 완료!
UserVitalSign 데이터:
         NO  WORKDATE  USERCODE  ISWEAR  HEARTBEAT  TEMPERATURE  \
0  1722304  20240801        -1       0          0          0.0   
1  1722305  20240801        -1       0          0          0.0   
2  1722306  20240801        -1       0          0          0.0   
3  1722307  20240801        -1       0          0          0.0   
4  1722308  20240801        -1       0          0          0.0   

   OUTSIDETEMPERATURE  LATITUDE  LONGITUDE  DEVICEBATTERY  \
0                   0       0.0        0.0            100   
1                   0       0.0        0.0            100   
2                   0       0.0        0.0             -1   
3                   0       0.0        0.0            100   
4                   0       0.0        0.0            100   

                VITALDATE             REGISTERDATE               MODIFYDATE  \
0 2024-08-01 01:17:28.823  2024-08-01 01:17:28.579  2024-08-01 01:17:28.579   
1 2024-08-01 02:28:02.210  2024-08-01 02:28

### `UserGyro` 데이터의 `WORKDATE`가 `VITALDATE`와 일치하지 않는 문제를 해결하기 위해 `VITALDATE` 날짜를 기준으로 `WORKDATE`를 재정의. `WORKDATE`가 `VITALDATE`의 날짜(연월일)와 일치하도록 수정

In [7]:
import pandas as pd
from tqdm import tqdm

# 1. 데이터 불러오기
gyro = pd.read_csv(r"C:\AI\원본 데이터\원본\UserGyro\BiscoDB_UserGyro-20240801.csv")

# 컬럼명 설정
gyro.columns = ["NO", "WORKDATE", "USERCODE", "X", "Y", "Z", "VITALDATE", "REGISTERDATE"]

# 2. 날짜 및 시간 변환
# VITALDATE를 datetime 형식으로 변환하고 변환 실패한 값은 제거
gyro['VITALDATE'] = pd.to_datetime(gyro['VITALDATE'], errors='coerce')
gyro.dropna(subset=['VITALDATE'], inplace=True)

# 3. WORKDATE 수정
# VITALDATE의 날짜를 사용해 WORKDATE를 갱신
# VITALDATE의 연월일을 'YYYYMMDD' 형식으로 추출하여 WORKDATE로 설정
gyro['WORKDATE'] = gyro['VITALDATE'].dt.strftime('%Y%m%d')

# 4. 데이터 오름차순 정렬
# VITALDATE를 기준으로 오름차순 정렬하여 과거부터 최신 순서대로 정렬
gyro.sort_values(by='VITALDATE', inplace=True)

# 5. 결과 확인 및 저장
print("UserGyro 데이터 예시:\n", gyro.head())

# 정제된 데이터 저장
gyro.to_csv("Cleaned_UserGyro.csv", index=False)
print("정제된 UserGyro 데이터 저장 완료!")

UserGyro 데이터 예시:
               NO  WORKDATE  USERCODE         X         Y         Z  \
773462  54456642  20240730        46 -0.290772 -0.069639 -0.182038   
773461  54456643  20240730        46  0.337198  0.179594  0.147829   
773460  54456644  20240730        46 -0.190590  0.024435  0.332311   
773459  54456645  20240730        46  0.161268  0.032987  0.453262   
773458  54456646  20240730        46  0.118508  0.097738  0.492357   

                        VITALDATE                REGISTERDATE  
773462 2024-07-30 10:50:00.970684  2024-08-02 07:24:24.260952  
773461 2024-07-30 10:50:01.130875  2024-08-02 07:24:24.260952  
773460 2024-07-30 10:50:01.289972  2024-08-02 07:24:24.260952  
773459 2024-07-30 10:50:01.451762  2024-08-02 07:24:24.260952  
773458 2024-07-30 10:50:01.625562  2024-08-02 07:24:24.260952  
정제된 UserGyro 데이터 저장 완료!


### `UserLocationLog` 데이터에서 `CHECKTIME`의 날짜를 기준으로 `WORKDATE`를 갱신하여 `WORKDATE`와 `CHECKTIME`의 날짜가 일치하도록 수정

In [9]:
import pandas as pd
from tqdm import tqdm

# 데이터 불러오기
location_log = pd.read_csv(r"C:\AI\원본 데이터\원본\UserLocationLog.csv")

# 컬럼명 설정
location_log.columns = ["NO", "WORKDATE", "USERCODE", "LATITUDE", "LONGITUDE", "COORDINATE", "SATELLITE", 
                        "ACCURACY", "ALTITUDE", "VERTICALACCURACY", "BEARING", "BEARINGACCURACY", 
                        "DIRECTION", "SPEED", "SPEEDACCURACY", "UNIXTIME", "CHECKTIME", 
                        "LOCATIONPROVIDER", "ELAPSEDREALTIMEAGEMILLIS", "NETWORKOPERATOR", 
                        "NETWORKPROVIDER", "REGISTERDATE", "REGISTEREMPCODE", "MODIFYDATE", "MODIFYEMPCODE"]

# 불필요한 열 제거 (필요시)
location_log.drop(columns=["COORDINATE", "REGISTEREMPCODE", "MODIFYEMPCODE", "NETWORKOPERATOR", "NETWORKPROVIDER"], inplace=True)

# CHECKTIME을 datetime 형식으로 변환하고 변환 실패한 값은 제거
location_log['CHECKTIME'] = pd.to_datetime(location_log['CHECKTIME'], errors='coerce')
location_log.dropna(subset=['CHECKTIME'], inplace=True)

# CHECKTIME의 날짜를 'YYYYMMDD' 형식으로 변환하여 WORKDATE에 반영
location_log['WORKDATE'] = location_log['CHECKTIME'].dt.strftime('%Y%m%d')

# 오름차순 정렬 (CHECKTIME 기준)
location_log.sort_values(by='CHECKTIME', inplace=True)

# 결과 확인 및 저장
print("UserLocationLog 데이터 예시:\n", location_log.head())

# 정제된 데이터 저장
location_log.to_csv("Cleaned_UserLocationLog.csv", index=False)
print("정제된 UserLocationLog 데이터 저장 완료!")

UserLocationLog 데이터 예시:
            NO  WORKDATE  USERCODE   LATITUDE   LONGITUDE  SATELLITE  ACCURACY  \
1798  9254323  20240731        17  35.166052  129.056285          0      20.0   
1799  9254324  20240731        17  35.166052  129.056286          0      20.0   
1800  9254325  20240731        17  35.166052  129.056286          0      20.0   
1801  9254326  20240731        17  35.166052  129.056286          0      20.0   
1802  9254327  20240731        17  35.166052  129.056286          0      20.0   

       ALTITUDE  VERTICALACCURACY  BEARING  BEARINGACCURACY  DIRECTION  \
1798  48.299999           1.00649      0.0              0.0          0   
1799  48.299999           1.00571      0.0              0.0          0   
1800  48.299999           1.00262      0.0              0.0          0   
1801  48.299999           1.00868      0.0              0.0          0   
1802  48.299999           1.01297      0.0              0.0          0   

         SPEED  SPEEDACCURACY       UNIXTIM

### `UserGyro` 여러 CSV 파일을 통합하고, `WORKDATE`를 `VITALDATE`에 맞추어 갱신한 뒤 하나의 파일로 저장합

In [10]:
import pandas as pd
import os
from tqdm import tqdm

# 1. UserGyro 데이터 파일이 저장된 폴더 경로 지정
folder_path = r'C:\AI\원본 데이터\원본\UserGyro'  # UserGyro 데이터가 저장된 폴더 경로로 변경

# 2. 모든 CSV 파일 불러오기 및 데이터 통합
all_data = []  # 모든 데이터를 저장할 리스트
first_file = True  # 첫 파일 여부를 확인하기 위한 플래그

for file_name in tqdm(os.listdir(folder_path)):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        
        # 첫 번째 파일에서만 컬럼명 설정
        if first_file:
            gyro_data = pd.read_csv(file_path)
            gyro_data.columns = ["NO", "WORKDATE", "USERCODE", "X", "Y", "Z", "VITALDATE", "REGISTERDATE"]
            first_file = False  # 이후 파일에서는 컬럼명을 설정하지 않음
        else:
            gyro_data = pd.read_csv(file_path, header=None, names=["NO", "WORKDATE", "USERCODE", "X", "Y", "Z", "VITALDATE", "REGISTERDATE"])
        
        # 각 파일의 데이터를 리스트에 추가
        all_data.append(gyro_data)

# 리스트에 담긴 모든 데이터를 하나의 DataFrame으로 통합
gyro = pd.concat(all_data, ignore_index=True)
print("데이터 통합 완료!")

# 3. VITALDATE를 기준으로 WORKDATE 갱신
# VITALDATE를 datetime 형식으로 변환
gyro['VITALDATE'] = pd.to_datetime(gyro['VITALDATE'], errors='coerce')

# VITALDATE가 변환되지 않은 행 삭제
gyro.dropna(subset=['VITALDATE'], inplace=True)

# VITALDATE의 날짜 부분을 'YYYYMMDD' 형식으로 설정하여 WORKDATE 갱신
gyro['WORKDATE'] = gyro['VITALDATE'].dt.strftime('%Y%m%d')

# 4. VITALDATE 기준으로 오름차순 정렬
gyro.sort_values(by='VITALDATE', inplace=True)

# 5. 통합 및 정제된 데이터 저장
gyro.to_csv("Cleaned_All_UserGyro.csv", index=False)
print("모든 날짜 통합 및 정제된 UserGyro 데이터 저장 완료!")

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:29<00:00,  1.39s/it]


데이터 통합 완료!
모든 날짜 통합 및 정제된 UserGyro 데이터 저장 완료!
